In [9]:
import os
from merge_sensor_data_step1 import load_station_data, load_township_polygons, add_township_info, process_years
import pandas as pd

# 設定輸出 txt 檔名
base_path = "../dataset/sensor"
missing_locations_all_years_path = "missing_locations_all_years.txt"

In [10]:
def txt_to_dict(file_path):
    data_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:  # 過濾空白行
                continue
            # 使用冒號分割 key 與 value
            key, value_str = line.split(":", 1)
            # 去除左右空白並移除中括號
            value_str = value_str.strip().strip("[]")
            values = []
            for num in value_str.split(","):
                num = num.strip()
                # 檢查是否為 "None"
                if num == "None":
                    values.append(None)
                else:
                    try:
                        values.append(float(num))
                    except ValueError:
                        print(f"警告：無法將 {num} 轉換成 float")
            data_dict[key] = values
    return data_dict

In [11]:
def fill_missing_lon_lat(year, missing_locations_all_years):
    file_name = f"flood_data_{year}_merged.csv"
    df = pd.read_csv(os.path.join(base_path, file_name))

    # 建立經度與緯度的 mapping 字典
    longitude_map = {k: v[0] for k, v in missing_locations_all_years.items()}
    latitude_map = {k: v[1] for k, v in missing_locations_all_years.items()}

    # 利用 map 方法，若 PQ_fullname 存在於 mapping 中，則用 mapping 的值，否則保留原值
    df['Longitude'] = df['PQ_fullname'].map(longitude_map).fillna(df['Longitude'])
    df['Latitude'] = df['PQ_fullname'].map(latitude_map).fillna(df['Latitude'])
    
    polygons = load_township_polygons()
    df = add_township_info(df, polygons)
    df = df.dropna(subset=["TOWNNAME"])
    
    df.loc[df['value'] < 0, 'value'] = 0
    return df

In [12]:
missing_locations_all_years = txt_to_dict(os.path.join(base_path, missing_locations_all_years_path))

for year in range(2019, 2026):
    merged_df = process_years(base_path, year)
    # if you already have the df, then
    # merged_df = pd.read_csv(os.path.join(base_path, f"flood_data_{year}_merged.csv"))
    merged_df = fill_missing_lon_lat(year, missing_locations_all_years)
    merged_df.to_csv(os.path.join(base_path, f"flood_data_{year}_merged_filled.csv"))